In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

In [12]:
def close_sign_in_if_open(driver):
    botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
    if botton_signup.is_displayed():
        botton_signup.click()

In [13]:
service = Service(executable_path='../chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f'https://www.investing.com/economic-calendar/')
time.sleep(1)

botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
botton_cookies.click()
time.sleep(1)

close_sign_in_if_open(driver)
botton_yesterday = driver.find_element(
    By.XPATH, '//*[@id="timeFrame_yesterday"]'
)
botton_yesterday.click()
time.sleep(1)

In [14]:
close_sign_in_if_open(driver)
table = driver.find_element(
    By.XPATH, '//*[@class="eCalNew eCalMainNew ecoCalCont"]'
)
dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
df_raw = dfs[0]
if df_raw.columns[0] is not str: # Fix for ocasional multi-index
    df_raw.columns = [c[0] for c in df_raw.columns]
df_raw

,Time,Cur.,Imp.,Event,Actual,Forecast,Previous,Unnamed: 7_level_0,Unnamed: 8_level_0
0,"Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023","Sunday, November 5, 2023"
1,18:50,JPY,NaN,Monetary Policy Meeting Minutes,NaN,NaN,NaN,NaN,NaN
2,19:00,AUD,NaN,MI Inflation Gauge (MoM),-0.1%,NaN,0.0%,NaN,NaN
3,19:00,NZD,NaN,ANZ Commodity Price Index (MoM),2.9%,NaN,1.4%,NaN,NaN
4,19:30,AUD,NaN,ANZ Job Advertisements (MoM),-3.0%,NaN,-0.5%,NaN,NaN
5,19:30,JPY,NaN,au Jibun Bank Japan Services PMI (Oct),51.6,51.1,51.1,NaN,NaN
6,20:05,JPY,NaN,BOJ Gov Ueda Speaks,NaN,NaN,NaN,NaN,NaN


In [15]:
df_filtered = df_raw[~df_raw['Cur.'].isna()] # Exclude holidays
df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows
df_filtered = df_filtered[df_filtered['Imp.'].isna()] # Remove rows with dates

close_sign_in_if_open(driver)
# Read countries' names from flags (important for EUR)
flags = driver.find_elements(
    By.XPATH, '//*[@class="left flagCur noWrap"]/span'
)
countries = []
for flag in flags:
    countries.append(flag.get_attribute('title'))

df_filtered['Country'] = countries
df_filtered = df_filtered[~df_filtered['Actual'].isna()] # Get only indicators

df = df_filtered[['Time', 'Country', 'Cur.', 'Event',
                  'Actual', 'Forecast', 'Previous']]
df.index = pd.to_datetime(df_filtered['Date'])
df.index.name = None
df

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-11-05,19:00,Australia,AUD,MI Inflation Gauge (MoM),-0.1%,NaN,0.0%
2023-11-05,19:00,New Zealand,NZD,ANZ Commodity Price Index (MoM),2.9%,NaN,1.4%
2023-11-05,19:30,Australia,AUD,ANZ Job Advertisements (MoM),-3.0%,NaN,-0.5%
2023-11-05,19:30,Japan,JPY,au Jibun Bank Japan Services PMI (Oct),51.6,51.1,51.1


In [16]:
driver.close()